In [1]:
import pandas as pd
import random
import numpy as np
import pickle
import json
import copy
from matplotlib import pyplot as plt
from scipy.io import savemat
import math
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import shutil  # 导入移动模块
import collections
import scipy
import scipy.stats as stats

In [2]:
ll_path = "../"
rank_name = ll_path + 'rank.txt'

rank_array = np.loadtxt(rank_name, dtype=np.float32, delimiter=',')
rank = int(rank_array[2])


filename = ll_path + 'data' + str(rank) + '/'
dot = '.npy'
mac_kind = int(rank_array[0])
sev_kind = int(rank_array[1])

mac_sev_name = 'mac_sev' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
mac_trf_name = 'mac_trf' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
sev_trf_name = 'sev_trf' + '_' + str(sev_kind) + '_' + str(rank)  # 后面可以补充字符串说明
sev_trf_out_name = 'sev_trf_out' + '_' + \
    str(sev_kind) + '_' + str(rank)  # 后面可以补充字符串说明
sev_trf_in_name = 'sev_trf_in' + '_' + \
    str(sev_kind) + '_' + str(rank)  # 后面可以补充字符串说明
sev_cpu_name = 'sev_cpu' + '_' + str(sev_kind) + '_' + str(rank)
sev_mem_name = 'sev_mem' + '_' + str(sev_kind) + '_' + str(rank)
mac_cpu_name = 'mac_cpu' + '_' + str(mac_kind) + '_' + str(rank)
mac_mem_name = 'mac_mem' + '_' + str(mac_kind) + '_' + str(rank)
total_cpu_name = 'total_cpu' + '_' + str(mac_kind) + '_' + str(rank)
total_mem_name = 'total_mem' + '_' + str(mac_kind) + '_' + str(rank)
mac_sev_cpu_name = 'mac_sev_cpu' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
mac_sev_mem_name = 'mac_sev_mem' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
now_all_name = 'now_all' + '_' + str(mac_kind) + '_' + str(rank)

trf_cpu_mem = np.load(filename + now_all_name + dot)
mac_sev_cpu = np.load(filename + mac_sev_cpu_name + dot)
mac_sev_mem = np.load(filename + mac_sev_mem_name + dot)
mac_sev = np.load(filename + mac_sev_name + dot)
sev_trf = np.load(filename + sev_trf_name + dot)
sev_trf_out = np.load(filename + sev_trf_out_name + dot)
sev_trf_in = np.load(filename + sev_trf_in_name + dot)

s_cpu = np.load(filename + sev_cpu_name + dot)
s_mem = np.load(filename + sev_mem_name + dot)
stop = np.zeros(mac_sev.shape[0]).reshape((mac_sev.shape[0], 1))
trf_cpu_mem = np.concatenate((trf_cpu_mem, stop), axis=1)

In [3]:
p = 0.3
a = 3
name = 'BT_under_BS_a' + str(a) + '_' + str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank) + '_' + 'r1 + r2' + '_' + str(p) + 'para3.csv'

In [4]:
name

'BT_under_BS_a3_20_100_2_r1 + r2_0.3para3.csv'

In [5]:
def CosineDistance(x, y):
    x = np.array(x)
    y = np.array(y)
    s = (np.linalg.norm(x)*np.linalg.norm(y))
    if s == 0:
        return 0
    return np.dot(x,y)/s

In [6]:
aco_mac_sev = np.loadtxt('./' + str(mac_kind) + '_' + str(sev_kind) + '/' + 'ACO.csv', delimiter=',')
mns_mac_sev = np.loadtxt('./' + str(mac_kind) + '_' + str(sev_kind) + '/' + 'MNS.csv', delimiter=',')
best_mac_sev = np.loadtxt('./' + str(mac_kind) + '_' + str(sev_kind) + '/' + name, delimiter=',')

In [7]:
exp = mac_sev.sum(axis = 0)

In [8]:
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p,M)+0.5*scipy.stats.entropy(q, M)

In [11]:
def mean_cos(x, y):
    cos_list = []
    for i in range(x.shape[0]):
        cos_i = CosineDistance(x[i], exp)
        if sum(x[i]) != 0:
            cos_list.append(cos_i)
    print(cos_list)
    print(min(cos_list))
    print(max(cos_list))
    return np.mean(cos_list)

In [12]:
aco_cos = mean_cos(aco_mac_sev, exp)
print("aco : {}".format(aco_cos))
mns_cos = mean_cos(mns_mac_sev, exp)
print("mns : {}".format(mns_cos))
rl_cos = mean_cos(best_mac_sev, exp)
print("rl : {}".format(rl_cos))

[0.7768715542254755, 0.23834573289160382, 0.7281861146038454, 0.6750543904329576, 0.2177317632846298, 0.7639961450135656, 0.7122497347993809, 0.7100167471046318, 0.6549990207103595, 0.7690681482551228, 0.7129188315641461]
0.2177317632846298
0.7768715542254755
aco : 0.6326761984441563
[0.8499613255958877, 0.8942180229097187, 0.9200389972864381, 0.8514873279351614, 0.8639898011479041, 0.8434682763137386, 0.802814619508918, 0.8302036969336293, 0.762940017540281, 0.7554142558572158, 0.5432880676205676, 0.8021335212967012, 0.706972525959939]
0.5432880676205676
0.9200389972864381
mns : 0.8020715735312385
[0.6340504687757739, 0.9098631781333778, 0.8891503978571572, 0.8432477354160138, 0.8872150385975267, 0.8799491279431519, 0.8920442306405236, 0.8790513693481167, 0.8873709918766323, 0.8659849458775333, 0.8371656102000193]
0.6340504687757739
0.9098631781333778
rl : 0.8550084631514387


## 数据生成API

In [3]:
#5台物理机的话 也就是30种服务 
#然后还要保证其能关机 所以数据的密集程度不能太高 要低一些 比如有一台8% 有一台可能50% 有一台可能90%

#这样的话 就先生成服务 再计算MAC CPU
#虚拟机的大小 CPU  10 - 100 MEM 是 5 - 80
#CPU 的占有率 和 MEM 的占有率 不一样 一样的显得造的不太随机
#每台物理机的占有率 目前我这种造数据方式 是跟着MAC CPU大小相关 可能与服务的个数不那么相关 就是说可能出现 物理机大 服务数多 占用率低
#但也需要注意一下 别关不了机 所以尽可能的关占用率低的 

#为了让他有0 得有40%的概率让他随机出0
#每台物理机最多放20 - 90个 最多也就100个了 CPU是1% - 3% 按理说 一个物理机做多放 30 - 100个SEV

In [5]:
#5 - 80 5台物理机是完全一样的 
#10 - 100 
#20 - 150



In [3]:
seq = {5 : 1, 10 : 2, 20 : 4, 40 : 5, 50 : 5, 60 : 5,  80 : 5, 100 : 5, 120 : 6, 150 : 6}

In [4]:
#最小公倍数
def lcm(x, y):
    a = int(x)
    b = int(y)
    return a * b // math.gcd(a, b)

def gen_mac_sev(t_mac_kind, t_sev_kind, t_sev_num_low, t_sev_num_high, rp):
    zero = 0
    t_mac_sev = np.zeros((t_mac_kind, t_sev_kind))
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            if random.random() < 0.5:
                t_mac_sev[i][j] = 0
            else:
                t_mac_sev[i][j] = random.randint(t_sev_num_low, t_sev_num_high)
    
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            if random.random() < rp:
                t_mac_sev[i][j] = 0

    per_kinds = list(t_mac_sev.sum(axis = 0))
    for i in range(len(per_kinds)):
        if zero == per_kinds[i]:
            i_random = random.randint(zero, t_mac_sev.shape[0] - 1)
            t_mac_sev[i_random][i] = random.randint(max(t_sev_num_low, 1), t_sev_num_high)
    return t_mac_sev


def gen_sev_cpu_mem(t_sev_kind, t_c_l, t_c_r, t_m_l, t_m_r):
    t_sev_cpu = np.zeros(t_sev_kind)
    t_sev_mem = np.zeros(t_sev_kind)
    for i in range(t_sev_kind):
        t_sev_cpu[i] = random.randint(t_c_l, t_c_r)
        t_sev_mem[i] = random.randint(t_m_l, t_m_r)
    return t_sev_cpu, t_sev_mem
    
def gen_sev_trf(t_sev_kind, t_trf_low, t_trf_high):
    t_sev_trf = np.zeros((t_sev_kind, t_sev_kind))
    for i in range(t_sev_kind - 1):
        for j in range(i + 1, t_sev_kind):
            if random.random() < 0.4:
                t_sev_trf[i][j] = 0
            else:
                t_sev_trf[i][j] = int(random.randint(t_trf_low, t_trf_high))
            #sev_trf[j][i] = sev_trf[i][j]
    return t_sev_trf

def compute_mac_cpu_and_mem(t_sev_cpu, t_sev_mem, t_mac_kind, t_sev_kind, t_mac_sev):
    t_mac_cpu = np.zeros(t_mac_kind)
    t_mac_mem = np.zeros(t_mac_kind)
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            t_mac_cpu[i] += t_mac_sev[i][j] * t_sev_cpu[j]
            t_mac_mem[i] += t_mac_sev[i][j] * t_sev_mem[j]
    return t_mac_cpu, t_mac_mem

def compute_mac_trf(t_mac_kind, t_sev_kind, t_mac_sev, t_sev_trf_out, t_sev_trf_in):
    t_mac_trf = np.zeros(t_mac_kind)
    for i in range(t_mac_kind):
        for j in range(t_sev_kind - 1):
            for k in range(j + 1, t_sev_kind):
                t_mac_trf[i] += min(t_mac_sev[i][j] * t_sev_trf_out[j][k], t_mac_sev[i][k] * t_sev_trf_in[j][k])
    return t_mac_trf
#%%%%%%%%%%%%%%%%%
def gen_total_cpu_and_mem(t_mac_sev, t_mac_cpu, t_mac_mem, t_mac_kind):
    t_p = np.zeros((t_mac_kind, 2))
    t_total_cpu = np.zeros(t_mac_kind)
    t_total_mem = np.zeros(t_mac_kind)

    for i in range(t_mac_kind):
        i_sev_num = sum(t_mac_sev[i])
        # num / 100 认为是最多都能放100个
        #p[i][0] = i_sev_num / one_mac_max_sev_num
        #p[i][1] = p[i][0]
        t_p[i][0] = random.uniform(0.6, 0.8)
        t_p[i][1] =  t_p[i][0] 
        t_total_cpu[i] = int(t_mac_cpu[i] / t_p[i][0]) 
        t_total_mem[i] = int(t_mac_mem[i] / t_p[i][1])
        #利用率认为基本一致 但是 CPU和MEM的差距很大
        t_p[i][0] = t_mac_cpu[i] / t_total_cpu[i]
        t_p[i][1] = t_mac_mem[i] / t_total_mem[i] 
    return t_p, t_total_cpu, t_total_mem

def gtotal_and_now_cpumem(test_cpu, t_total_cpu, test_mem, t_total_mem, t_mac_trf):
    return np.stack((t_mac_trf, test_cpu, t_total_cpu, test_cpu/t_total_cpu, test_mem, t_total_mem, test_mem/t_total_mem), axis = 1)


def gen_sev_trf_out_and_in(t_sev_trf, t_mac_sev, t_sev_kind, func):
    sev_trf_out = np.zeros((t_sev_kind, t_sev_kind))
    sev_trf_in = np.zeros((t_sev_kind, t_sev_kind))
    per_kinds_demand = t_mac_sev.sum(axis = 0)
    for i in range(t_sev_kind - 1):
        for j in range(i + 1, t_sev_kind):
            d_i = copy.deepcopy(per_kinds_demand[i])
            #print('Testdi')
            #print((i, d_i))
            d_j = copy.deepcopy(per_kinds_demand[j])
            #print('Testdj')
            #print((j, d_j))
            # 28, 3, 5 -- > 28 / 15 = 1 15, 3-5, 5-3
            #i --> j
            d = func(d_i, d_j)
            #print('d, sev_yrfij')
            #print(d)
            #print(sev_trf[i][j])
            t_sev_trf[i][j] = int(t_sev_trf[i][j] / d) * d
            sev_trf_out[i][j] = t_sev_trf[i][j] / d_i
            sev_trf_in[i][j] = t_sev_trf[i][j] / d_j
    return t_sev_trf, sev_trf_out, sev_trf_in

def gen_ms_cpu_mem(t_mac_kind, t_sev_kind, t_mac_sev, t_sev_cpu, t_sev_mem):
    mac_sev_cpu = np.zeros((t_mac_kind, t_sev_kind))
    mac_sev_mem = np.zeros((t_mac_kind, t_sev_kind))
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            if t_mac_sev[i][j] > 0:
                mac_sev_cpu[i][j] = t_sev_cpu[j]
                mac_sev_mem[i][j] = t_sev_mem[j]
    return mac_sev_cpu, mac_sev_mem

def test(t_mac_sev, t_sev_cpu, t_sev_mem, t_total_cpu, t_total_mem, p_com):
    m_c_m = np.zeros((t_mac_sev.shape[0], 2))
    t_c_m = np.zeros((t_mac_sev.shape[0], 2))
    p = np.zeros((t_mac_sev.shape[0], 2))
    for i in range(t_mac_sev.shape[0]):
        t_c_m[i][0] = t_total_cpu[i]
        t_c_m[i][1] = t_total_mem[i]
        for j in range(t_mac_sev.shape[1]):
            m_c_m[i][0] += t_mac_sev[i][j] * t_sev_cpu[j]
            m_c_m[i][1] += t_mac_sev[i][j] * t_sev_mem[j]

    p = m_c_m / t_c_m 
    print('利用率相等吗')
    print(np.allclose(p, p_com))
    #print('cpu mem')
    #print(total_cpu)
    #print(total_mem)
    print('放得下吗')
    temp = m_c_m < t_c_m
    print(sum(sum(temp)) == t_mac_sev.shape[0] * 2)
    print(p.sum(axis = 0) / t_mac_sev.shape[0])
    #return p == p_com

    

In [5]:
#把这个参数改一遍 别使用全局变量

In [6]:
#mac_kind == mac_num

In [7]:
def regen(t_c, t_m, m_c, m_m, mac_num, s):
    p = np.zeros((mac_num, 2))

    kinds = s[mac_num]#mac有几种
    num = int(mac_num / kinds)#每种多少个
    # 100  5种 每种20个

    t_c_new = copy.deepcopy(t_c)
    t_m_new = copy.deepcopy(t_m)
    t_c_new.sort(reverse = True)
    t_m_new.sort(reverse = True)

    for i in range(kinds):#第0种 1 2 3 4 
        for j in range(num):#第0 - 19
            t = i * num + j
            t_c[t] = t_c_new[i]
            t_m[t] = t_m_new[i]
            p[t][0] = m_c[t] / t_c[t]
            p[t][1] = m_m[t] / t_m[t]

    return p, t_c, t_m
    

In [8]:
def gen_data(num, mac_kind, sev_kind, sev_num_low, sev_num_high, trf_low, trf_high, c_l, c_r, m_l, m_r, the_rp):
    the_name = np.zeros(3)
    the_name[0] = mac_kind
    the_name[1] = sev_kind
    the_name[2] = num
    
    
    for i in range(num):
        mac_sev = gen_mac_sev(mac_kind, sev_kind, sev_num_low, sev_num_high, the_rp)
        sev_cpu, sev_mem = gen_sev_cpu_mem(sev_kind, c_l, c_r, m_l, m_r)
        sev_trf_old = gen_sev_trf(sev_kind, trf_low, trf_high)
        sev_trf_new, sev_trf_out, sev_trf_in = gen_sev_trf_out_and_in(sev_trf_old, mac_sev, sev_kind, lcm)

        mac_cpu, mac_mem = compute_mac_cpu_and_mem(sev_cpu, sev_mem, mac_kind, sev_kind, mac_sev)
        p, total_cpu, total_mem = gen_total_cpu_and_mem(mac_sev, mac_cpu, mac_mem, mac_kind)
        #####################################
        p, total_cpu, total_mem = regen(list(total_cpu), list(total_mem), mac_cpu, mac_mem, mac_kind, seq)
        ####################################
        mac_sev_cpu, mac_sev_mem = gen_ms_cpu_mem(mac_kind, sev_kind, mac_sev, sev_cpu, sev_mem)

        mac_trf = compute_mac_trf(mac_kind, sev_kind, mac_sev, sev_trf_out, sev_trf_in)

        now_all = gtotal_and_now_cpumem(mac_cpu, total_cpu, mac_mem, total_mem, mac_trf)
        print(i + 1)
        test(mac_sev, sev_cpu, sev_mem, total_cpu, total_mem, p)

        #print(p) 
        #print(sum(mac_sev))
        print(sum(sum(mac_sev)))
        print()

        mac_sev_name = 'mac_sev' + '_' + str(mac_kind) + '_' + str(sev_kind) + '_' + str(i + 1)
        mac_trf_name = 'mac_trf' + '_' + str(mac_kind) + '_' + str(sev_kind) + '_' + str(i + 1)
        sev_trf_name = 'sev_trf' + '_' + str(sev_kind) + '_' + str(i + 1) #后面可以补充字符串说明
        sev_trf_out_name = 'sev_trf_out' + '_' + str(sev_kind) + '_' + str(i + 1) #后面可以补充字符串说明
        sev_trf_in_name = 'sev_trf_in' + '_' + str(sev_kind) + '_' + str(i + 1) #后面可以补充字符串说明
        sev_cpu_name = 'sev_cpu' + '_' + str(sev_kind) + '_' + str(i + 1)
        sev_mem_name = 'sev_mem' + '_' + str(sev_kind) + '_' + str(i + 1)
        mac_cpu_name = 'mac_cpu' + '_' + str(mac_kind) + '_' + str(i + 1)
        mac_mem_name = 'mac_mem' + '_' + str(mac_kind) + '_' + str(i + 1)
        total_cpu_name = 'total_cpu' + '_' + str(mac_kind) + '_' + str(i + 1)
        total_mem_name = 'total_mem' + '_' + str(mac_kind) + '_' + str(i + 1)
        mac_sev_cpu_name = 'mac_sev_cpu' + '_' + str(mac_kind) + '_' + str(sev_kind) + '_' + str(i + 1)
        mac_sev_mem_name = 'mac_sev_mem' + '_' + str(mac_kind) + '_' + str(sev_kind) + '_' + str(i + 1)
        now_all_name = 'now_all' + '_' + str(mac_kind) + '_' + str(i + 1)

        r_list = ['r1', 'r2', 'r3', 'r1 + r2', 'r1 + r3', 'r2 + r3', 'r1 + r2 + r3']

        #5_50/1/
        path = str(mac_kind) + '_' + str(sev_kind) + '/' + str(i + 1) + '/'
        dot = '.npy'
        dot_csv = '.csv'
        
        #5_50/1/data1
        filename = path + 'data' + str(i + 1) + '/'
        os.makedirs(filename)
        #5_50/1/5_50_1_NoExp
        Noexp = path + 'NoExp'
        os.makedirs(Noexp)
        Yesexp = path + 'YesExp'
        os.makedirs(Yesexp)
        # 移动目标文件夹的根目录
        movabs_path = './5_40/1/NoExp/'
        # 移动文件夹的根目录
        rawabs_path = Noexp + '/'
        # 移动操作

        #这个是不是224的
        shutil.copytree(movabs_path + "r3", rawabs_path + 'r3')
        shutil.copytree(movabs_path + "r1+r2", rawabs_path + "r1+r2")
        shutil.copy('./5_40/para1.txt', str(mac_kind) + '_' + str(sev_kind) + '/para1.txt' )
        shutil.copy('./5_40/para3.txt', str(mac_kind) + '_' + str(sev_kind) + '/para3.txt' )
        #shutil.copy('../' + 'Exp/5_80/run_a1.py', str(mac_kind) + '_' + str(sev_kind) + '/run_a1.py' )
        rank_array = np.zeros(3)
        rank_array[0] = int(mac_kind)
        rank_array[1] = int(sev_kind)
        rank_array[2] = int(i + 1)
        np.savetxt(path + 'rank.txt', rank_array , fmt='%d')

        np.save(filename + mac_sev_name + dot, mac_sev)
        np.save(filename + mac_trf_name + dot, mac_trf)
        np.save(filename + sev_trf_name + dot, sev_trf_new)
        np.save(filename + sev_trf_out_name + dot, sev_trf_out)
        np.save(filename + sev_trf_in_name + dot, sev_trf_in)
        np.save(filename + sev_cpu_name + dot, sev_cpu)
        np.save(filename + sev_mem_name + dot, sev_mem)
        np.save(filename + mac_cpu_name + dot, mac_cpu)
        np.save(filename + mac_mem_name + dot, mac_mem)
        np.save(filename + total_cpu_name + dot, total_cpu)
        np.save(filename + total_mem_name + dot, total_mem)
        np.save(filename + mac_sev_cpu_name + dot, mac_sev_cpu)
        np.save(filename + mac_sev_mem_name + dot, mac_sev_mem)
        np.save(filename + now_all_name + dot, now_all)

        csv_name = path + 'data_csv' + str(mac_kind) + '_' + str(sev_kind) + '_' +str(i + 1) + '/'
        os.makedirs(csv_name)
        np.savetxt(csv_name + mac_sev_name + dot_csv, mac_sev, delimiter="," )
        np.savetxt(csv_name + mac_trf_name + dot_csv, mac_trf, delimiter="," )
        np.savetxt(csv_name + sev_trf_name + dot_csv, sev_trf_new, delimiter="," )
        np.savetxt(csv_name + sev_trf_out_name + dot_csv, sev_trf_out, delimiter="," )
        np.savetxt(csv_name + sev_trf_in_name + dot_csv, sev_trf_in, delimiter="," )
        np.savetxt(csv_name + sev_cpu_name + dot_csv, sev_cpu, delimiter="," )
        np.savetxt(csv_name + sev_mem_name + dot_csv, sev_mem, delimiter="," )
        np.savetxt(csv_name + total_cpu_name + dot_csv, total_cpu, delimiter="," )
        np.savetxt(csv_name + total_mem_name + dot_csv, total_mem, delimiter="," )
        np.savetxt(csv_name + now_all_name + dot_csv, now_all, delimiter="," )

        #总的东西 
        per_kind_mac_num = np.zeros((seq[mac_kind], 1))
        for a in range(per_kind_mac_num.shape[0]):
            per_kind_mac_num[a][0] = int(mac_kind / seq[mac_kind])
        per_mac_c_m = np.zeros((seq[mac_kind] , 2))
        for a in range(per_mac_c_m.shape[0]):
            per_mac_c_m[a][0] = total_cpu[int(a * per_kind_mac_num[a][0])]
            per_mac_c_m[a][1] = total_mem[int(a * per_kind_mac_num[a][0])]

        vm_demand = np.zeros((sev_kind, 2))
        for a in range(sev_kind):
            vm_demand[a][0] = sev_cpu[a]
            vm_demand[a][1] = sev_mem[a]

        per_sev_num = (mac_sev.sum(axis = 0)).reshape((sev_kind, 1))
        #要给测试的 总cpu mem是不是一样大的
        test_c = sum(mac_cpu)
        test_m = sum(mac_mem)
        
        t_c = 0
        t_m = 0
        for a in range(vm_demand.shape[0]):
            t_c += vm_demand[a][0] * per_sev_num[a]
            t_m += vm_demand[a][1] * per_sev_num[a]
        print('总量算对了吗')
        print(t_c == test_c)
        print(t_m == test_m)
        print('以上')

        savemat(path + str(mac_kind) + '_' + str(sev_kind) + '_' + str(i + 1) + ".mat", 
                {"whole_mac_nums" : mac_kind, "mac_kinds": seq[mac_kind], "mac_kinds_per_num": per_kind_mac_num, 
                "mac_kinds_per_total_cpu_mem": per_mac_c_m, 
                "vm_total_num": sum(sum(mac_sev)), 'vm_kinds' : sev_kind, 
                'vm_kinds_per_num' : per_sev_num, 
                'vm_kinds_per_cpu_mem' :vm_demand, 'sev_trf_out':sev_trf_out, 'sev_trf_in' : sev_trf_in, 'sev_trf' : sev_trf_new})  
        
    np.savetxt(str(mac_kind) + '_' + str(sev_kind) + '/' + 'name.txt', the_name , fmt='%d')

In [12]:
mac_kind = 20
sev_kind = 100
rank = 0
#想生成的数据范围
sev_num_low = 1
sev_num_high = 3
trf_low = 100
trf_high = 1000
c_l = 5
c_r = 100
m_l = 10
m_r = 80
rp = 0.48
num = 3
gen_data(num, mac_kind, sev_kind, sev_num_low, sev_num_high, trf_low, trf_high, c_l, c_r, m_l, m_r, rp)

1
利用率相等吗
True
放得下吗
True
[0.52829989 0.51344542]
1000.0

总量算对了吗
[ True]
[ True]
以上
2
利用率相等吗
True
放得下吗
True
[0.50777506 0.49071515]
998.0

总量算对了吗
[ True]
[ True]
以上
3
利用率相等吗
True
放得下吗
True
[0.52745289 0.53558695]
1092.0

总量算对了吗
[ True]
[ True]
以上


In [9]:
mac_kind = 40
sev_kind = 150
rank = 0
#想生成的数据范围
sev_num_low = 1
sev_num_high = 3
trf_low = 100
trf_high = 1000
c_l = 5
c_r = 100
m_l = 10
m_r = 80
rp = 0.74
num = 3
gen_data(num, mac_kind, sev_kind, sev_num_low, sev_num_high, trf_low, trf_high, c_l, c_r, m_l, m_r, rp)

1
利用率相等吗
True
放得下吗
True
[0.47369258 0.46447865]
1581.0

总量算对了吗
[ True]
[ True]
以上
2
利用率相等吗
True
放得下吗
True
[0.48510102 0.50969963]
1626.0

总量算对了吗
[ True]
[ True]
以上
3
利用率相等吗
True
放得下吗
True
[0.49393136 0.4737389 ]
1569.0

总量算对了吗
[ True]
[ True]
以上


In [16]:
mac_kind = 60
sev_kind = 200
rank = 0
#想生成的数据范围
sev_num_low = 1
sev_num_high = 3
trf_low = 100
trf_high = 1000
c_l = 5
c_r = 100
m_l = 10
m_r = 80
rp = 0.83
num = 3
gen_data(num, mac_kind, sev_kind, sev_num_low, sev_num_high, trf_low, trf_high, c_l, c_r, m_l, m_r, rp)

1
利用率相等吗
True
放得下吗
True
[0.4374433  0.44049386]
2010.0

总量算对了吗
[ True]
[ True]
以上
2
利用率相等吗
True
放得下吗
True
[0.39959782 0.44387926]
1956.0

总量算对了吗
[ True]
[ True]
以上
3
利用率相等吗
True
放得下吗
True
[0.43895229 0.46917472]
1957.0

总量算对了吗
[ True]
[ True]
以上


## 一次调度数据处理

In [231]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import collections
import copy
import matplotlib.pyplot as plt
import os

#from src import Envn
#from modules import ActorCritic, CNNandDense
#import rl_utils
from tqdm import tqdm
# 5_50/1/5_50_No/r1/reward,a1

lll_path = "../"
rank_name = lll_path + 'rank.txt'

rank_array = np.loadtxt(rank_name, dtype=np.float32, delimiter=',')
rank = int(rank_array[2])

reward_kind_name = 'reward.txt'
reward_kind = np.loadtxt(rank_name, dtype=np.float32, delimiter=',')

filename = lll_path + 'data' + str(rank) + '/'
dot = '.npy'
mac_kind = int(rank_array[0])
sev_kind = int(rank_array[1])

OSError: ../../../rank.txt not found.

In [ ]:
the_path = ''

In [101]:
vm_per_num = mac_sev.sum(axis = 0)

In [103]:
#总的东西 
savemat("" +'_' + str(mac_kinds) + '_' + str(sev_kinds) + ".mat", {"whole_mac_nums" : whole_mac_nums, "mac_kinds": mac_kinds, "mac_kinds_per_num": per_kind_mac_num.reshape((mac_kinds, 1)), "mac_kinds_per_total_cpu_mem": per_mac_c_m, 
                            "vm_total_num": sum(per_sev_kinds), 'vm_kinds' : sev_kinds, 'vm_kinds_per_num' :per_sev_kinds.reshape((per_sev_kinds.shape[0], 1)), 'vm_kinds_per_cpu_mem' :vm_d, 'sev_trf':sev_trf_1 })

3959.0

In [380]:
mac_kinds_name = []
for mac in mac_index:
    c = cpu_m[mac]
    m = mem_m[mac]
    if (c, m) not in mac_kinds_name:
        mac_kinds_name.append((c, m))
mac_kinds = len(mac_kinds_name)


In [383]:
mac_kinds_name

[(10913.0, 5297.0),
 (11029.0, 5353.0),
 (11025.0, 21493.0),
 (11154.0, 21744.0),
 (10947.0, 5313.0),
 (11201.0, 21835.0),
 (10946.0, 21339.0),
 (10871.0, 5277.0),
 (10889.0, 10599.0),
 (11060.0, 14392.0),
 (11185.0, 21804.0),
 (10932.0, 21312.0),
 (10914.0, 28405.0)]

In [382]:
mac_kinds

13

In [384]:
mac_total_name = []
for mac in mac_index:
    c = cpu_m[mac]
    m = mem_m[mac]
    mac_total_name.append((c, m))
mac_total = len(mac_total_name)

In [385]:
mac_total_name #40 2

[(10913.0, 5297.0),
 (10913.0, 5297.0),
 (10913.0, 5297.0),
 (10913.0, 5297.0),
 (10913.0, 5297.0),
 (11029.0, 5353.0),
 (11029.0, 5353.0),
 (11029.0, 5353.0),
 (11025.0, 21493.0),
 (11154.0, 21744.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (10947.0, 5313.0),
 (11201.0, 21835.0),
 (11201.0, 21835.0),
 (11201.0, 21835.0),
 (11201.0, 21835.0),
 (11201.0, 21835.0),
 (11201.0, 21835.0),
 (11201.0, 21835.0),
 (10946.0, 21339.0),
 (10946.0, 21339.0),
 (10871.0, 5277.0),
 (10889.0, 10599.0),
 (11060.0, 14392.0),
 (11185.0, 21804.0),
 (11185.0, 21804.0),
 (11185.0, 21804.0),
 (11185.0, 21804.0),
 (11185.0, 21804.0),
 (10932.0, 21312.0),
 (10932.0, 21312.0),
 (10932.0, 21312.0),
 (10914.0, 28405.0)]

In [386]:
mac_per_kinds_num = np.zeros(mac_kinds)
for i, cm in enumerate(mac_kinds_name):
    mac_per_kinds_num[i] = mac_total_name.count(cm)

In [388]:
sum(mac_per_kinds_num)

40.0

In [389]:
for i in range(len(mac_kinds_name)):
    mac_kinds_name[i] = np.array(mac_kinds_name[i])

In [390]:
mac_kinds_cpu_mem = np.array(mac_kinds_name)

In [107]:
mac_kinds_cpu_mem = np.array((total_cpu[0], total_mem[0]))

In [108]:
mac_kinds_cpu_mem

array([87221.09479938, 42973.60320267])

In [110]:
savemat("./data/data_20_40/data_20_40.mat", {"mac_total_num" : mac_kind, "mac_kinds_num": 1, "mac_kinds_per_num": (np.array(mac_kind)).reshape((1, 1)), "mac_kinds_per_cpu_mem": mac_kinds_cpu_mem , 
                                                                    "vm_total_num": sum(vm_per_num), 'vm_kinds_num' : sev_kind, 'vm_kinds_per_num' : vm_per_num.reshape((sev_kind, 1)), 'vm_kinds_per_cpu_mem' :vm_att, 'sev_trf':sev_trf })

## 数据处理

In [6]:
df = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/df.csv",index_col = 0,header=0)

In [7]:
df

,Service,RequestCPU,RequestMem,ContainerList,CompatibleMachines
0,Service0,0.000008,0.000013,"['Container0', 'Container1', 'Container2', 'Co...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
1,Service1,0.000029,0.000037,"['Container20', 'Container21', 'Container22', ...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
2,Service2,0.000014,0.000009,"['Container33', 'Container34', 'Container35', ...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
3,Service3,0.000029,0.000009,"['Container63', 'Container64', 'Container65', ...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
4,Service4,0.000017,0.000002,"['Container112', 'Container113', 'Container114...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
...,...,...,...,...,...
5899,Service5899,0.000029,0.000036,"['Container25632', 'Container25633']","['0.0.1.201', '0.0.3.195', '0.0.3.196']"
5900,Service5900,0.000015,0.000005,"['Container25634', 'Container25635', 'Containe...","['0.0.1.201', '0.0.3.195', '0.0.3.196']"
5901,Service5901,0.000008,0.000005,['Container25637'],"['0.0.1.201', '0.0.3.195', '0.0.3.196']"
5902,Service5902,0.000029,0.000009,['Container25638'],"['0.0.1.201', '0.0.3.195', '0.0.3.196']"


In [8]:
cpu_m = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/cpu_m_big.csv",index_col = 0,header=0)
cpu_m = cpu_m['TotalCPU']

In [9]:
cpu_m = cpu_m * 10

In [225]:
cpu_m_ini = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/cpu_m_big.csv",index_col = 0,header=0)
cpu_m_ini = cpu_m_ini['TotalCPU']
cpu_m_ini = cpu_m_ini * 10

In [226]:
cpu_m_ini

MachineIP
0.0.0.0      21940.336525
0.0.0.1      21940.336525
0.0.0.2      21940.336525
0.0.0.3      21940.336525
0.0.0.4      21940.336525
                 ...     
0.0.3.204     5436.293327
0.0.3.205     5510.912876
0.0.3.206     5471.600770
0.0.3.207     5471.600770
0.0.3.208     5487.173684
Name: TotalCPU, Length: 977, dtype: float64

In [10]:
cpu_m

MachineIP
0.0.0.0      21940.336525
0.0.0.1      21940.336525
0.0.0.2      21940.336525
0.0.0.3      21940.336525
0.0.0.4      21940.336525
                 ...     
0.0.3.204     5436.293327
0.0.3.205     5510.912876
0.0.3.206     5471.600770
0.0.3.207     5471.600770
0.0.3.208     5487.173684
Name: TotalCPU, Length: 977, dtype: float64

In [11]:
for i in cpu_m.index:
    cpu_m[i] = int(cpu_m[i])

In [230]:
cpu_m

MachineIP
0.0.0.0      21940.0
0.0.0.1      21940.0
0.0.0.2      21940.0
0.0.0.3      21940.0
0.0.0.4      21940.0
              ...   
0.0.3.204     5436.0
0.0.3.205     5510.0
0.0.3.206     5471.0
0.0.3.207     5471.0
0.0.3.208     5487.0
Name: TotalCPU, Length: 977, dtype: float64

In [13]:
mem_m = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/mem_m_big.csv",index_col = 0,header=0)
mem_m = mem_m['TotalMem']

In [14]:
mem_m = mem_m * 10

In [227]:
mem_m_ini = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/mem_m_big.csv",index_col = 0,header=0)
mem_m_ini = mem_m_ini['TotalMem']
mem_m_ini = mem_m_ini * 10

In [228]:
mem_m_ini

MachineIP
0.0.0.0      14276.112946
0.0.0.1      14276.112946
0.0.0.2      14276.112946
0.0.0.3      14276.112946
0.0.0.4      14276.112946
                 ...     
0.0.3.204     5277.847979
0.0.3.205     5350.292678
0.0.3.206     5312.126357
0.0.3.207     5312.126357
0.0.3.208     5327.245385
Name: TotalMem, Length: 977, dtype: float64

In [15]:
mem_m

MachineIP
0.0.0.0      14276.112946
0.0.0.1      14276.112946
0.0.0.2      14276.112946
0.0.0.3      14276.112946
0.0.0.4      14276.112946
                 ...     
0.0.3.204     5277.847979
0.0.3.205     5350.292678
0.0.3.206     5312.126357
0.0.3.207     5312.126357
0.0.3.208     5327.245385
Name: TotalMem, Length: 977, dtype: float64

In [16]:
for i in mem_m.index:
    mem_m[i] = int(mem_m[i])

In [229]:
mem_m

MachineIP
0.0.0.0      14276.0
0.0.0.1      14276.0
0.0.0.2      14276.0
0.0.0.3      14276.0
0.0.0.4      14276.0
              ...   
0.0.3.204     5277.0
0.0.3.205     5350.0
0.0.3.206     5312.0
0.0.3.207     5312.0
0.0.3.208     5327.0
Name: TotalMem, Length: 977, dtype: float64

In [18]:
cpu_s = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/cpu_s_big.csv",index_col = 0,header=0)

In [19]:
cpu_s = cpu_s['RequestCPU'] * 10

In [231]:
cpu_s_ini = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/cpu_s_big.csv",index_col = 0,header=0)
cpu_s_ini = cpu_s_ini['RequestCPU'] * 10

In [232]:
cpu_s_ini

Service
Service0        83.785630
Service1       294.993174
Service2       144.810923
Service3       289.641653
Service4       172.487116
                  ...    
Service5899    285.094948
Service5900    146.986588
Service5901     82.115760
Service5902    291.572056
Service5903    141.872291
Name: RequestCPU, Length: 5904, dtype: float64

In [20]:
cpu_s

Service
Service0        83.785630
Service1       294.993174
Service2       144.810923
Service3       289.641653
Service4       172.487116
                  ...    
Service5899    285.094948
Service5900    146.986588
Service5901     82.115760
Service5902    291.572056
Service5903    141.872291
Name: RequestCPU, Length: 5904, dtype: float64

In [21]:
for i,j in enumerate(cpu_s):
    if i < 100:
        print(j)

83.78562959874982
294.9931735690817
144.81092306646485
289.64165282991877
172.4871157038462
146.0029294771777
289.424065380533
387.44213319910494
289.6795063610444
158.61896224425743
138.4086866089969
100.65945673851964
142.77724451359884
291.3503101015904
72.42363518715032
158.15496166010323
143.09243183863427
142.2487473754953
147.7728967689161
79.52362354618843
289.4640292783355
71.83364656615636
291.959974540934
84.70263936005452
153.18766269977812
152.16750955345725
305.0943686451699
586.8213454956933
150.52616302248785
287.5736770773492
174.60970111627387
584.2323866852151
140.90933503451572
586.3425170292549
292.32737424022724
293.2565196399264
174.6564770208728
147.2352462249686
292.9380459925412
146.67217906229195
294.76711759516536
143.21415304217982
293.7070354930471
82.45344092969509
295.08184085186423
165.8382015368224
142.90250954636457
145.7181849020374
84.47802076679592
145.49676550293754
155.8286131380788
291.9384732376738
145.5511093145976
446.4572474614272
186.597615

In [22]:
#Service5449
for i in cpu_s.index:
    cpu_s[i] = int(cpu_s[i])

In [236]:
cpu_s

Service
Service0        83.0
Service1       294.0
Service2       144.0
Service3       289.0
Service4       172.0
               ...  
Service5899    285.0
Service5900    146.0
Service5901     82.0
Service5902    291.0
Service5903    141.0
Name: RequestCPU, Length: 5904, dtype: float64

In [24]:
#强制让他为1 因为他是0 看一下最后不会溢出吧
cpu_s[5449] = 1

In [25]:
for i in cpu_s.index:
    if(cpu_s[i] == 0):
        print(i)

In [26]:
mem_s = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/mem_s_big.csv",index_col = 0,header=0)

In [27]:
mem_s = mem_s['RequestMem'] * 10

In [233]:
mem_s_ini = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/mem_s_big.csv",index_col = 0,header=0)
mem_s_ini = mem_s_ini['RequestMem'] * 10

In [234]:
mem_s_ini

Service
Service0       134.352036
Service1       374.385750
Service2        91.892204
Service3        92.989783
Service4        24.575453
                  ...    
Service5899    361.823579
Service5900     46.636405
Service5901     46.318422
Service5902     92.510974
Service5903     90.027446
Name: RequestMem, Length: 5904, dtype: float64

In [28]:
for i in mem_s.index:
    mem_s[i] = int(mem_s[i])

In [29]:
#[706,1005,4873,5449]==0
mem0 = [706,1005,4873,5449]
for i in mem0:
    mem_s[i] =1

In [235]:
mem_s

Service
Service0       134.0
Service1       374.0
Service2        91.0
Service3        92.0
Service4        24.0
               ...  
Service5899    361.0
Service5900     46.0
Service5901     46.0
Service5902     92.0
Service5903     90.0
Name: RequestMem, Length: 5904, dtype: float64

In [31]:
for i in mem_s.index:
    if(mem_s[i] == 0):
        print(i)

## 实际数据生成 只用了第一个文件 有的服务为0 记住他 不用他 并且扩大了xx倍 扩大了 应该是1e7

In [32]:
#Service5449
#有cpu不占用但占用内存的 不过不打紧 去掉它
#mem [706,1005,4873,5449]==0
not_use = [706,1005,4873,5449]

In [33]:
cpu_m

MachineIP
0.0.0.0      21940.0
0.0.0.1      21940.0
0.0.0.2      21940.0
0.0.0.3      21940.0
0.0.0.4      21940.0
              ...   
0.0.3.204     5436.0
0.0.3.205     5510.0
0.0.3.206     5471.0
0.0.3.207     5471.0
0.0.3.208     5487.0
Name: TotalCPU, Length: 977, dtype: float64

In [34]:
mem_m

MachineIP
0.0.0.0      14276.0
0.0.0.1      14276.0
0.0.0.2      14276.0
0.0.0.3      14276.0
0.0.0.4      14276.0
              ...   
0.0.3.204     5277.0
0.0.3.205     5350.0
0.0.3.206     5312.0
0.0.3.207     5312.0
0.0.3.208     5327.0
Name: TotalMem, Length: 977, dtype: float64

In [35]:
cpu_s #第一个数据是不是有必须放得 也就是sev的最后一列其实没用是吧？？？

Service
Service0        83.0
Service1       294.0
Service2       144.0
Service3       289.0
Service4       172.0
               ...  
Service5899    285.0
Service5900    146.0
Service5901     82.0
Service5902    291.0
Service5903    141.0
Name: RequestCPU, Length: 5904, dtype: float64

In [36]:
mem_s

Service
Service0       134.0
Service1       374.0
Service2        91.0
Service3        92.0
Service4        24.0
               ...  
Service5899    361.0
Service5900     46.0
Service5901     46.0
Service5902     92.0
Service5903     90.0
Name: RequestMem, Length: 5904, dtype: float64

In [37]:
df = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/df.csv",index_col = 0,header=0)

In [38]:
df

,Service,RequestCPU,RequestMem,ContainerList,CompatibleMachines
0,Service0,0.000008,0.000013,"['Container0', 'Container1', 'Container2', 'Co...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
1,Service1,0.000029,0.000037,"['Container20', 'Container21', 'Container22', ...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
2,Service2,0.000014,0.000009,"['Container33', 'Container34', 'Container35', ...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
3,Service3,0.000029,0.000009,"['Container63', 'Container64', 'Container65', ...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
4,Service4,0.000017,0.000002,"['Container112', 'Container113', 'Container114...","['0.0.0.0', '0.0.0.1', '0.0.0.2', '0.0.0.3', '..."
...,...,...,...,...,...
5899,Service5899,0.000029,0.000036,"['Container25632', 'Container25633']","['0.0.1.201', '0.0.3.195', '0.0.3.196']"
5900,Service5900,0.000015,0.000005,"['Container25634', 'Container25635', 'Containe...","['0.0.1.201', '0.0.3.195', '0.0.3.196']"
5901,Service5901,0.000008,0.000005,['Container25637'],"['0.0.1.201', '0.0.3.195', '0.0.3.196']"
5902,Service5902,0.000029,0.000009,['Container25638'],"['0.0.1.201', '0.0.3.195', '0.0.3.196']"


In [39]:
dfm = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/dfm.csv",index_col = 0,header=0)

In [40]:
dfm

,MachineIP,TotalCPU,TotalMem,InitialDeployingContainers
0,0.0.0.0,0.002194,0.001428,"['Container0', 'Container20', 'Container33', '..."
1,0.0.0.1,0.002194,0.001428,"['Container333', 'Container334', 'Container335..."
2,0.0.0.2,0.002194,0.001428,"['Container21', 'Container337', 'Container338'..."
3,0.0.0.3,0.002194,0.001428,"['Container22', 'Container64', 'Container115',..."
4,0.0.0.4,0.002194,0.001428,"['Container1', 'Container65', 'Container66', '..."
...,...,...,...,...
972,0.0.3.204,0.000544,0.000528,['Container24758']
973,0.0.3.205,0.000551,0.000535,"['Container14022', 'Container14027', 'Containe..."
974,0.0.3.206,0.000547,0.000531,"['Container3054', 'Container3055', 'Container3..."
975,0.0.3.207,0.000547,0.000531,"['Container3058', 'Container3059', 'Container3..."


In [41]:
new_dfm = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/new_dfm.csv",index_col = 0,header=0)

In [42]:
new_dfm

,MachineIP,TotalCPU,TotalMem,InitialDeployingContainers,Ini_service
0,0.0.0.0,0.002194,0.001428,"['Container0', 'Container20', 'Container33', '...","['Service0', 'Service1', 'Service2', 'Service3..."
1,0.0.0.1,0.002194,0.001428,"['Container333', 'Container334', 'Container335...","['Service7', 'Service7', 'Service7', 'Service7..."
2,0.0.0.2,0.002194,0.001428,"['Container21', 'Container337', 'Container338'...","['Service1', 'Service7', 'Service7', 'Service7..."
3,0.0.0.3,0.002194,0.001428,"['Container22', 'Container64', 'Container115',...","['Service1', 'Service3', 'Service4', 'Service4..."
4,0.0.0.4,0.002194,0.001428,"['Container1', 'Container65', 'Container66', '...","['Service0', 'Service3', 'Service3', 'Service4..."
...,...,...,...,...,...
972,0.0.3.204,0.000544,0.000528,['Container24758'],['Service5229']
973,0.0.3.205,0.000551,0.000535,"['Container14022', 'Container14027', 'Containe...","['Service607', 'Service608', 'Service612', 'Se..."
974,0.0.3.206,0.000547,0.000531,"['Container3054', 'Container3055', 'Container3...","['Service157', 'Service157', 'Service157', 'Se..."
975,0.0.3.207,0.000547,0.000531,"['Container3058', 'Container3059', 'Container3...","['Service157', 'Service157', 'Service157', 'Se..."


In [43]:
mac_ini_sev = new_dfm['Ini_service']

In [44]:
mac_ini_sev

0      ['Service0', 'Service1', 'Service2', 'Service3...
1      ['Service7', 'Service7', 'Service7', 'Service7...
2      ['Service1', 'Service7', 'Service7', 'Service7...
3      ['Service1', 'Service3', 'Service4', 'Service4...
4      ['Service0', 'Service3', 'Service3', 'Service4...
                             ...                        
972                                      ['Service5229']
973    ['Service607', 'Service608', 'Service612', 'Se...
974    ['Service157', 'Service157', 'Service157', 'Se...
975    ['Service157', 'Service157', 'Service157', 'Se...
976    ['Service0', 'Service2', 'Service4', 'Service9...
Name: Ini_service, Length: 977, dtype: object

In [45]:
for i in mac_ini_sev.index:
    mac_ini_sev.loc[i] = eval(mac_ini_sev.loc[i])

C:\Users\28322\AppData\Local\Temp\ipykernel_12616\3870891682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mac_ini_sev.loc[i] = eval(mac_ini_sev.loc[i])


In [46]:
mac_ini_sev

0      [Service0, Service1, Service2, Service3, Servi...
1      [Service7, Service7, Service7, Service7, Servi...
2      [Service1, Service7, Service7, Service7, Servi...
3      [Service1, Service3, Service4, Service4, Servi...
4      [Service0, Service3, Service3, Service4, Servi...
                             ...                        
972                                        [Service5229]
973    [Service607, Service608, Service612, Service61...
974    [Service157, Service157, Service157, Service15...
975    [Service157, Service157, Service157, Service15...
976    [Service0, Service2, Service4, Service9, Servi...
Name: Ini_service, Length: 977, dtype: object

In [118]:
#先统计一下 有多少种mac
mac_kinds_list = []
c_m = []
for c,m in zip(cpu_m, mem_m):
    c_m.append((c, m))
    if (c, m) not in mac_kinds_list:
        mac_kinds_list.append((c, m))

In [119]:
mac_kinds_num = []
for i in mac_kinds_list:
    i_num = c_m.count(i)
    mac_kinds_num.append(i_num)

In [120]:
sum(mac_kinds_num)

977

In [121]:
len(mac_kinds_num)

122

In [122]:
mac_kinds = [(i[0], i[1], j) for i,j in zip(mac_kinds_list, mac_kinds_num)]

In [171]:
mac_cpu_mem_sev_new = []
mac_kinds_sev_kinds_new = []
mac_kinds_num_new = []
changed = []
for mk, i in enumerate(mac_cpu_mem_sev):
    if i[2] >= 5 and i[3] > 120:
        l1 = int(i[2] / 5)
        l2 = int(i[2] / 5)
        l3 = int(i[2] / 5)
        l4 = int(i[2] / 5)
        l5 = i[2] - l1 - l2 - l3 - l4

        #第mk种物理机的索引编号
        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        mac_kinds_sev_kinds_new.append(len(kinds)) 
        mac_cpu_mem_sev_new.append((i[0], i[1], l1, len(kinds))) 
        mac_kinds_num_new.append(l1)  
        changed.append((i[0], i[1], l1, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1
        l = r
        r = l + l2
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        mac_kinds_sev_kinds_new.append(len(kinds))
        mac_cpu_mem_sev_new.append((i[0], i[1], l2, len(kinds)))   
        mac_kinds_num_new.append(l2)  
        changed.append((i[0], i[1], l2, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1 + l2
        l = r
        r = l + l3
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        mac_kinds_sev_kinds_new.append(len(kinds))
        mac_cpu_mem_sev_new.append((i[0], i[1], l3, len(kinds)))   
        mac_kinds_num_new.append(l3)  
        changed.append((i[0], i[1], l3, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1 + l2 + l3
        l = r
        r = l + l4
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        mac_kinds_sev_kinds_new.append(len(kinds))
        mac_cpu_mem_sev_new.append((i[0], i[1], l4, len(kinds)))   
        mac_kinds_num_new.append(l4)  
        changed.append((i[0], i[1], l4, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1 + l2 + l3 + l4
        l = r
        r = l + l4
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        mac_kinds_sev_kinds_new.append(len(kinds))
        mac_cpu_mem_sev_new.append((i[0], i[1], l5, len(kinds)))   
        mac_kinds_num_new.append(l5)  
        changed.append((i[0], i[1], l5, len(kinds)))
    else:
        l = sum(mac_kinds_num[ : mk]) 
        r = l + mac_kinds_num[mk]
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        mac_kinds_sev_kinds_new.append(len(kinds))
        mac_cpu_mem_sev_new.append((i[0], i[1], mac_kinds_num[mk], len(kinds)))   
        mac_kinds_num_new.append(mac_kinds_num[mk])  

change = []
for i in changed:
    index = mac_cpu_mem_sev_new.index(i)
    change.append((i[0], i[1], i[2], i[3], index))

In [328]:
m_c_m_s_n = []
m_k_s_k_n = []
m_k_n_n = []
change_newd_new = []
for mk, i in enumerate(mac_cpu_mem_sev):
    if i[2] >= 5 and i[3] > 120:
        l1 = int(i[2] / 5)
        l2 = int(i[2] / 5)
        l3 = int(i[2] / 5)
        l4 = int(i[2] / 5)
        l5 = i[2] - l1 - l2 - l3 - l4

        #第mk种物理机的索引编号
        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        m_k_s_k_n.append(len(kinds)) 
        m_c_m_s_n.append((i[0], i[1], l1, len(kinds))) 
        m_k_n_n.append(l1)  
        change_newd_new.append((i[0], i[1], l1, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1
        l = r
        r = l + l2
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        m_k_s_k_n.append(len(kinds))
        m_c_m_s_n.append((i[0], i[1], l2, len(kinds)))   
        m_k_n_n.append(l2)  
        change_newd_new.append((i[0], i[1], l2, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1 + l2
        l = r
        r = l + l3
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        m_k_s_k_n.append(len(kinds))
        m_c_m_s_n.append((i[0], i[1], l3, len(kinds)))   
        m_k_n_n.append(l3)  
        change_newd_new.append((i[0], i[1], l3, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1 + l2 + l3
        l = r
        r = l + l4
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        m_k_s_k_n.append(len(kinds))
        m_c_m_s_n.append((i[0], i[1], l4, len(kinds)))   
        m_k_n_n.append(l4)  
        change_newd_new.append((i[0], i[1], l4, len(kinds)))

        l = sum(mac_kinds_num[ : mk]) 
        r = l + l1 + l2 + l3 + l4
        l = r
        r = l + l4
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        m_k_s_k_n.append(len(kinds))
        m_c_m_s_n.append((i[0], i[1], l5, len(kinds)))   
        m_k_n_n.append(l5)  
        change_newd_new.append((i[0], i[1], l5, len(kinds)))
    else:
        l = sum(mac_kinds_num[ : mk]) 
        r = l + mac_kinds_num[mk]
        kinds = []
        for m_mk in range(l, r):
            for s in mac_ini_sev[m_mk]:
                if s not in kinds:
                    kinds.append(s)
        m_k_s_k_n.append(len(kinds))
        m_c_m_s_n.append((i[0], i[1], mac_kinds_num[mk], len(kinds)))   
        m_k_n_n.append(mac_kinds_num[mk])  

change_new = []
for i in change_newd_new:
    index = m_c_m_s_n.index(i)
    change_new.append((i[0], i[1], i[2], i[3], index))

In [174]:
mac_cpu_mem_sev_new

[(21940.0, 14276.0, 1, 76),
 (21940.0, 14276.0, 1, 2),
 (21940.0, 14276.0, 1, 14),
 (21940.0, 14276.0, 1, 30),
 (21940.0, 14276.0, 3, 77),
 (21936.0, 14273.0, 1, 2),
 (22594.0, 14701.0, 5, 9),
 (22269.0, 14490.0, 2, 24),
 (22269.0, 14490.0, 2, 83),
 (22269.0, 14490.0, 2, 80),
 (22269.0, 14490.0, 2, 69),
 (22269.0, 14490.0, 5, 51),
 (22561.0, 29359.0, 1, 41),
 (21955.0, 14285.0, 1, 7),
 (22434.0, 29195.0, 1, 6),
 (14825.0, 14441.0, 1, 28),
 (10869.0, 5276.0, 1, 6),
 (11042.0, 10749.0, 93, 6),
 (11223.0, 29210.0, 1, 3),
 (10940.0, 5310.0, 2, 61),
 (10940.0, 5310.0, 2, 36),
 (10940.0, 5310.0, 2, 34),
 (10940.0, 5310.0, 2, 22),
 (10940.0, 5310.0, 3, 16),
 (11241.0, 7314.0, 34, 133),
 (11241.0, 7314.0, 34, 139),
 (11241.0, 7314.0, 34, 134),
 (11241.0, 7314.0, 34, 146),
 (11241.0, 7314.0, 37, 141),
 (10904.0, 28380.0, 3, 77),
 (10986.0, 5333.0, 7, 161),
 (10986.0, 5333.0, 7, 177),
 (10986.0, 5333.0, 7, 148),
 (10986.0, 5333.0, 7, 180),
 (10986.0, 5333.0, 7, 178),
 (11220.0, 14602.0, 11, 5),


In [173]:
change

[(21940.0, 14276.0, 1, 76, 0),
 (21940.0, 14276.0, 1, 2, 1),
 (21940.0, 14276.0, 1, 14, 2),
 (21940.0, 14276.0, 1, 30, 3),
 (21940.0, 14276.0, 3, 77, 4),
 (22269.0, 14490.0, 2, 24, 7),
 (22269.0, 14490.0, 2, 83, 8),
 (22269.0, 14490.0, 2, 80, 9),
 (22269.0, 14490.0, 2, 69, 10),
 (22269.0, 14490.0, 5, 51, 11),
 (10940.0, 5310.0, 2, 61, 19),
 (10940.0, 5310.0, 2, 36, 20),
 (10940.0, 5310.0, 2, 34, 21),
 (10940.0, 5310.0, 2, 22, 22),
 (10940.0, 5310.0, 3, 16, 23),
 (11241.0, 7314.0, 34, 133, 24),
 (11241.0, 7314.0, 34, 139, 25),
 (11241.0, 7314.0, 34, 134, 26),
 (11241.0, 7314.0, 34, 146, 27),
 (11241.0, 7314.0, 37, 141, 28),
 (10986.0, 5333.0, 7, 161, 30),
 (10986.0, 5333.0, 7, 177, 31),
 (10986.0, 5333.0, 7, 148, 32),
 (10986.0, 5333.0, 7, 180, 33),
 (10986.0, 5333.0, 7, 178, 34),
 (10900.0, 14184.0, 3, 70, 37),
 (10900.0, 14184.0, 3, 79, 38),
 (10900.0, 14184.0, 3, 76, 39),
 (10900.0, 14184.0, 3, 68, 40),
 (10900.0, 14184.0, 4, 56, 41),
 (11055.0, 10761.0, 3, 87, 43),
 (11055.0, 10761.

In [133]:
mac_cpu_mem_sev

[(21940.0, 14276.0, 7, 154),
 (21936.0, 14273.0, 1, 2),
 (22594.0, 14701.0, 5, 9),
 (22269.0, 14490.0, 13, 217),
 (22561.0, 29359.0, 1, 41),
 (21955.0, 14285.0, 1, 7),
 (22434.0, 29195.0, 1, 6),
 (14825.0, 14441.0, 1, 28),
 (10869.0, 5276.0, 1, 6),
 (11042.0, 10749.0, 93, 6),
 (11223.0, 29210.0, 1, 3),
 (10940.0, 5310.0, 11, 127),
 (11241.0, 7314.0, 173, 313),
 (10904.0, 28380.0, 3, 77),
 (10986.0, 5333.0, 35, 430),
 (11220.0, 14602.0, 11, 5),
 (11055.0, 14386.0, 5, 20),
 (10900.0, 14184.0, 16, 162),
 (10939.0, 28471.0, 2, 50),
 (11055.0, 10761.0, 18, 226),
 (10913.0, 5297.0, 16, 1482),
 (11029.0, 5353.0, 3, 6),
 (11025.0, 21493.0, 1, 10),
 (11153.0, 21744.0, 1, 35),
 (10946.0, 5313.0, 9, 147),
 (11200.0, 21835.0, 7, 258),
 (10946.0, 21339.0, 2, 6),
 (10871.0, 5277.0, 1, 5),
 (10888.0, 10599.0, 1, 3),
 (11059.0, 14392.0, 1, 10),
 (11184.0, 21804.0, 5, 17),
 (10932.0, 21312.0, 3, 9),
 (10913.0, 28405.0, 2, 5),
 (11173.0, 29081.0, 1, 24),
 (11185.0, 5429.0, 5, 73),
 (10944.0, 10653.0, 1,

In [325]:
mac_kinds_num

[7,
 1,
 5,
 13,
 1,
 1,
 1,
 1,
 1,
 93,
 1,
 11,
 173,
 3,
 35,
 11,
 5,
 16,
 2,
 18,
 16,
 3,
 1,
 1,
 9,
 7,
 2,
 1,
 1,
 1,
 5,
 3,
 2,
 1,
 5,
 1,
 4,
 6,
 14,
 1,
 1,
 2,
 2,
 1,
 19,
 2,
 26,
 8,
 9,
 4,
 25,
 2,
 4,
 2,
 3,
 21,
 1,
 18,
 2,
 3,
 1,
 52,
 1,
 1,
 6,
 3,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 9,
 3,
 10,
 2,
 17,
 8,
 10,
 4,
 3,
 7,
 3,
 7,
 2,
 1,
 5,
 5,
 3,
 1,
 2,
 1,
 2,
 10,
 1,
 20,
 1,
 30,
 9,
 2,
 16,
 3,
 3,
 13,
 9,
 1,
 1,
 6,
 4,
 5,
 10,
 2,
 2,
 2,
 3,
 1,
 1,
 1,
 1,
 2,
 1]

In [124]:
mac_kinds

[(21940.0, 14276.0, 7),
 (21936.0, 14273.0, 1),
 (22594.0, 14701.0, 5),
 (22269.0, 14490.0, 13),
 (22561.0, 29359.0, 1),
 (21955.0, 14285.0, 1),
 (22434.0, 29195.0, 1),
 (14825.0, 14441.0, 1),
 (10869.0, 5276.0, 1),
 (11042.0, 10749.0, 93),
 (11223.0, 29210.0, 1),
 (10940.0, 5310.0, 11),
 (11241.0, 7314.0, 173),
 (10904.0, 28380.0, 3),
 (10986.0, 5333.0, 35),
 (11220.0, 14602.0, 11),
 (11055.0, 14386.0, 5),
 (10900.0, 14184.0, 16),
 (10939.0, 28471.0, 2),
 (11055.0, 10761.0, 18),
 (10913.0, 5297.0, 16),
 (11029.0, 5353.0, 3),
 (11025.0, 21493.0, 1),
 (11153.0, 21744.0, 1),
 (10946.0, 5313.0, 9),
 (11200.0, 21835.0, 7),
 (10946.0, 21339.0, 2),
 (10871.0, 5277.0, 1),
 (10888.0, 10599.0, 1),
 (11059.0, 14392.0, 1),
 (11184.0, 21804.0, 5),
 (10932.0, 21312.0, 3),
 (10913.0, 28405.0, 2),
 (11173.0, 29081.0, 1),
 (11185.0, 5429.0, 5),
 (10944.0, 10653.0, 1),
 (11115.0, 21668.0, 4),
 (11215.0, 21863.0, 6),
 (10987.0, 5333.0, 14),
 (10861.0, 14135.0, 1),
 (11009.0, 21462.0, 1),
 (11134.0, 7244

In [83]:
#看一看每台物理机上的种类
each_mac_kinds = []
for m in range(sum(mac_kinds_num)):
    kinds = []
    for s in mac_ini_sev[m]:
        if s not in kinds:
            kinds.append(s)
    each_mac_kinds.append(len(kinds))

In [85]:
len(each_mac_kinds)

977

In [125]:
#看一看每种规格的物理机的sev种类
mac_kinds_sev_kinds = []
for mk in range(len(mac_kinds)):
    #第mk种物理机的索引编号
    l = sum(mac_kinds_num[ : mk]) 
    r = l + mac_kinds_num[mk]
    kinds = []
    for m_mk in range(l, r):
        for s in mac_ini_sev[m_mk]:
            if s not in kinds:
                kinds.append(s)
    mac_kinds_sev_kinds.append(len(kinds))

In [126]:
len(mac_kinds_sev_kinds)

122

In [90]:
mac_cpu_mem_sev = [(i[0], i[1], i[2], j) for i,j in zip(mac_kinds, mac_kinds_sev_kinds)]

In [92]:
len(mac_cpu_mem_sev)

122

In [93]:
mac_cpu_mem_sev

[(21940.0, 14276.0, 7, 154),
 (21936.0, 14273.0, 1, 2),
 (22594.0, 14701.0, 5, 9),
 (22269.0, 14490.0, 13, 217),
 (22561.0, 29359.0, 1, 41),
 (21955.0, 14285.0, 1, 7),
 (22434.0, 29195.0, 1, 6),
 (14825.0, 14441.0, 1, 28),
 (10869.0, 5276.0, 1, 6),
 (11042.0, 10749.0, 93, 6),
 (11223.0, 29210.0, 1, 3),
 (10940.0, 5310.0, 11, 127),
 (11241.0, 7314.0, 173, 313),
 (10904.0, 28380.0, 3, 77),
 (10986.0, 5333.0, 35, 430),
 (11220.0, 14602.0, 11, 5),
 (11055.0, 14386.0, 5, 20),
 (10900.0, 14184.0, 16, 162),
 (10939.0, 28471.0, 2, 50),
 (11055.0, 10761.0, 18, 226),
 (10913.0, 5297.0, 16, 1482),
 (11029.0, 5353.0, 3, 6),
 (11025.0, 21493.0, 1, 10),
 (11153.0, 21744.0, 1, 35),
 (10946.0, 5313.0, 9, 147),
 (11200.0, 21835.0, 7, 258),
 (10946.0, 21339.0, 2, 6),
 (10871.0, 5277.0, 1, 5),
 (10888.0, 10599.0, 1, 3),
 (11059.0, 14392.0, 1, 10),
 (11184.0, 21804.0, 5, 17),
 (10932.0, 21312.0, 3, 9),
 (10913.0, 28405.0, 2, 5),
 (11173.0, 29081.0, 1, 24),
 (11185.0, 5429.0, 5, 73),
 (10944.0, 10653.0, 1,

In [104]:
dft = pd.read_csv("C:/Users/28322/Desktop/OR_2023_VM/dataset/dft.csv",index_col = 0,header=0)
dft

new_dft = dft.set_index(['Service1','Service2'],drop=['Service1', 'Service2'])

dft_dict = dict({})

trf = []
for i in new_dft["Traffic"]:
    trf.append(i)
max_t = max(trf)
#print(trf)
#print(max_t)
for i, t in enumerate(trf):
    trf[i] = t / max_t * 1000

for k, i in enumerate(list(new_dft.index)):
    l = eval(i[0].strip("Service"))
    r = eval(i[1].strip("Service"))
    dft_dict.update({(l, r) : trf[k]})

In [105]:
dft_dict

{(2419, 1771): 4.298153090208228e-05,
 (2419, 4179): 4.298153090208228e-05,
 (2419, 4309): 4.298153090208228e-05,
 (2247, 4485): 0.058328073510073165,
 (4688, 3587): 3.7838987814221204e-07,
 (1755, 1034): 4.653487026587255e-05,
 (1755, 704): 9.30697405317451e-05,
 (1755, 3879): 4.653487026587255e-05,
 (4588, 1034): 2.3267435132936267e-05,
 (4588, 704): 4.653487026587255e-05,
 (4588, 3879): 2.3267435132936267e-05,
 (1918, 1034): 0.00011633717566468136,
 (1918, 704): 0.00023267435132936275,
 (1918, 3879): 0.00011633717566468136,
 (4348, 1034): 2.3267435132936267e-05,
 (4348, 704): 4.653487026587255e-05,
 (4348, 3879): 2.3267435132936267e-05,
 (675, 1034): 6.980230539880883e-05,
 (675, 704): 0.00013960461079761766,
 (675, 3879): 6.980230539880883e-05,
 (3845, 1034): 2.3267435132936267e-05,
 (3845, 704): 4.653487026587255e-05,
 (3845, 3879): 2.3267435132936267e-05,
 (1492, 1034): 4.653487026587255e-05,
 (1492, 704): 9.30697405317451e-05,
 (1492, 3879): 4.653487026587255e-05,
 (3672, 1034):

### 选服务和物理机

In [106]:
mac_ini_sev

0      [Service0, Service1, Service2, Service3, Servi...
1      [Service7, Service7, Service7, Service7, Servi...
2      [Service1, Service7, Service7, Service7, Servi...
3      [Service1, Service3, Service4, Service4, Servi...
4      [Service0, Service3, Service3, Service4, Servi...
                             ...                        
972                                        [Service5229]
973    [Service607, Service608, Service612, Service61...
974    [Service157, Service157, Service157, Service15...
975    [Service157, Service157, Service157, Service15...
976    [Service0, Service2, Service4, Service9, Servi...
Name: Ini_service, Length: 977, dtype: object

In [329]:
#原版注意

#函数内部不要使用全局变量
def gen_sev_l_r(l, r, t_mac_k_sev_k, t_mac_kinds_num, m_c_m_s):
    #t_mac_k_sev_k == mac_kinds_sev_kinds
    sev_l_r = [] #mac规格序号
    for i in range(len(t_mac_k_sev_k)):
        if l <= t_mac_k_sev_k[i] <= r:
            sev_l_r.append((i, t_mac_kinds_num[i], t_mac_k_sev_k[i], m_c_m_s[i][0], m_c_m_s[i][1]))
    return sev_l_r
    
def gen_mac(sevs_l_r, t_mac_kinds_num, t_not_use, t_x, t_mac_cpu_mem_sev, t_mac_ini_sev):
    t_i_l = random.sample(range(len(sevs_l_r)), t_x) #这是list index
    print(t_i_l)
    for i in t_i_l:
        print(sevs_l_r[i])

    for i in t_i_l:#i是list index
        t_i_k = sevs_l_r[i]#t_i_k mac_index
        print(t_mac_cpu_mem_sev[t_i_k[0]])

    macs = [sevs_l_r[i][0] for i in t_i_l]#mac_index

    whole_kinds = []
    for i in t_i_l:
        m_index = sevs_l_r[i][0]
        #第m种物理机的索引编号
        l = sum(t_mac_kinds_num[ : m_index]) 
        r = l + t_mac_kinds_num[m_index]
        for m_mk in range(l, r):
            for s in t_mac_ini_sev[m_mk]:
                if s not in whole_kinds:
                    whole_kinds.append(s)

    for i in range(len(whole_kinds)):
        whole_kinds[i] = eval(whole_kinds[i].strip('Service'))

    whole_kinds.sort()
    if_can_use = [i in whole_kinds for i in not_use]

    return macs, whole_kinds, len(whole_kinds), if_can_use

def vm_demand(t_sub_cpu, t_sub_mem, sev_kindsx):
    per_sev_kinds_c_m = np.zeros((sev_kindsx, 2))#每种sev
    for i in range(sev_kindsx):
        per_sev_kinds_c_m[i][0] = t_sub_cpu[i]
        per_sev_kinds_c_m[i][1] = t_sub_mem[i]
    return per_sev_kinds_c_m

#然后之后用计算的macsev＋demand算占用量 和 直接用cpu_s取int 用mac_ini_sev算服务对比

#写几部分函数
#判断当前能不能用 有没有not_use的元素 这个时候转为整数 不用string
#chosen_se要按大小顺序

#计算基本的参数 物理机种类 对应的个数 服务种类 对应的个数
def cmp_para(t_chosen_sevs, t_chosen_macs, t_mac_cpu_mem_sev):
    #mac_cpu_mem_sev (m_cpu, m_mem, m_num, s_kinds)

    ###mac
    x_mac_kinds = len(t_chosen_macs)
    t_whole_mac_nums = 0
    per_kind_mac_num = np.zeros(x_mac_kinds)

    for i, m in enumerate(t_chosen_macs):#(mac索引)
        per_kind_mac_num[i] = t_mac_cpu_mem_sev[m][2]
    t_whole_mac_nums = sum(per_kind_mac_num)
    
    ###sev 
    t_sev_kinds = len(t_chosen_sevs)

    return x_mac_kinds, int(t_whole_mac_nums), per_kind_mac_num, t_sev_kinds

#服务顺序的映射 #新服务索引的cpu mem
def sev_index_map(t_chosen_sevs, t_cpu_s, t_mem_s):
    #t_chosen_sevs.sort()
    sub_cpu = np.zeros((len(t_chosen_sevs)))
    sub_mem = np.zeros((len(t_chosen_sevs)))
    for s,i in zip(t_chosen_sevs, range(len(t_chosen_sevs))):
        sub_cpu[i] = int(t_cpu_s[s])
        sub_mem[i] = int(t_mem_s[s])
    return sub_cpu, sub_mem

#把初始分布矩阵求出来 
def gen_mac_sev(t_mac_cpu_mem_sev, t_mac_kinds_num, t_mac_ini_sev, t_sub_cpu, t_sub_mem, t_mac_kinds, t_whole_mac_nums, t_per_kind_mac_num, t_sev_kinds, t_chosen_macs, t_chosen_sevs):
    mac_sev = np.zeros((t_whole_mac_nums, t_sev_kinds))
    cnt = 0
    for i in range(t_mac_kinds):
        for j in range(int(t_per_kind_mac_num[i])):#第i种的第j个 
            #实际物理机m_i_s索引 
            #cm[i]是mac种类索引
            t_m_index = sum(t_mac_kinds_num[ : t_chosen_macs[i]]) + j
            for sev in t_mac_ini_sev[t_m_index]:
                #service1 不用set也不会重复计算某种服务
                s_cnt = t_mac_ini_sev[t_m_index].count(sev)
                s = eval(sev.strip('Service'))#实际索引
                #map 在此处的索引
                map_s_index = t_chosen_sevs.index(s)

                mac_sev[cnt][map_s_index] = s_cnt
            cnt += 1
    return mac_sev

#初始流量矩阵求出来
def cmp_trf(t_chosen_sevs, t_sev_kinds):
    sev_trf = np.zeros((t_sev_kinds, t_sev_kinds))
    #gen
    #100sev 0 - 98 i + 1 -- 99
    for i in range(t_sev_kinds - 1):
        for j in range(i + 1, t_sev_kinds):
            sev_trf[i][j] = random.randint(0, 1000)
    return sev_trf

#初始机内流量求出来 不用我算 一会再写
def cmp_mac_trf(whole_mac_nums, sev_trf, mac_cpu_mem_sev):
    pass
    
def total_cpu_mem(t_chosen_macs, t_mac_cpu_mem_sev, t_mac_kindsx):
    per_kind_mac_cpu_mem = np.zeros((t_mac_kindsx, 2))
    for i in range(per_kind_mac_cpu_mem.shape[0]):
        m = t_chosen_macs[i]
        per_kind_mac_cpu_mem[i][0] = int(t_mac_cpu_mem_sev[m][0])
        per_kind_mac_cpu_mem[i][1] = int(t_mac_cpu_mem_sev[m][1])
    return per_kind_mac_cpu_mem
    
def each_mac_total(t_whole_mac_numsx, t_per_kind_mac_num, t_mac_cpu_mem_sev, t_chosen_macs, t_mac_kindsx):
    each_mac_t_c_m = np.zeros((t_whole_mac_numsx, 2))
    cnt = 0
    for i in range(t_mac_kindsx):
        for j in range(int(t_per_kind_mac_num[i])):
            m = t_chosen_macs[i]
            each_mac_t_c_m[cnt][0] = int(t_mac_cpu_mem_sev[m][0])
            each_mac_t_c_m[cnt][1] = int(t_mac_cpu_mem_sev[m][1])
            cnt += 1
    return each_mac_t_c_m

#还有最终的验证 看看对不对
def test_truth(vm, t_mac_sev, t_sub_cpu, t_sub_mem, t_mac_ini_sev, t_cpu_s, t_mem_s, t_whole_mac_nums, t_chosen_macs, t_per_kind_mac_num, t_each_mac_t_c_mx, t_mac_kinds_num, per_sev_kinds_t):
    now_m_c_m = np.zeros((t_whole_mac_nums, 2))
    now_m_c_m1 = np.zeros((t_whole_mac_nums, 2))
    #是否符合实际
    for i in range(t_mac_sev.shape[0]):
        for j in range(t_mac_sev.shape[1]):
            now_m_c_m[i][0] += vm[j][0] * t_mac_sev[i][j]
            now_m_c_m[i][1] += vm[j][1] * t_mac_sev[i][j]

            now_m_c_m1[i][0] += t_sub_cpu[j] * t_mac_sev[i][j]
            now_m_c_m1[i][1] += t_sub_mem[j] * t_mac_sev[i][j]
    print('机子<MAX')
    print(now_m_c_m <= t_each_mac_t_c_mx)
    print('占有率')
    print(now_m_c_m / t_each_mac_t_c_mx)
    print('vm == sub')
    print(np.allclose(now_m_c_m, now_m_c_m1))
    #是否初始分布和原本的mac一样
    #看一看每种规格的物理机的sev种类 和 有流量的
    test_c_m = np.zeros((t_whole_mac_nums, 2))
    c = 0
    #c 是test序号
    for i, mk in enumerate(t_chosen_macs):
        #第mk种物理机的索引编号
        l = sum(t_mac_kinds_num[ : mk]) 
        r = l + t_mac_kinds_num[mk]
        print('是不是算对了 个数')
        print(r - l == int(t_per_kind_mac_num[i]))
        #m_mk是物理机实际编号 
        for m_mk, j in zip( range(l, r), range(int(t_per_kind_mac_num[i])) ):
            for s in t_mac_ini_sev[m_mk]:
                test_c_m[c][0] += int(t_cpu_s[s])
                test_c_m[c][1] += int(t_mem_s[s])
            c += 1 
    print(test_c_m)
    print(now_m_c_m)
    print('机子c m一不一样')
    print(np.allclose(test_c_m,now_m_c_m))

    print('机子数目一不一样')
    print(t_mac_sev.shape[0] == t_whole_mac_nums)

    t_c = 0
    t_m = 0
    for i in range(per_sev_kinds_t.shape[0]):
        num = per_sev_kinds_t[i]
        t_c += vm[i][0] * num
        t_m += vm[i][1] * num
    print('机子vm一不一样')
    print((now_m_c_m.sum(axis = 0)) == (t_c, t_m))
        
    return now_m_c_m, test_c_m

def half_sev_trf(t_sev_trf):
    sev_trft = copy.deepcopy(t_sev_trf)
    for i in range(sev_trft.shape[0] - 1):
        for j in range(i + 1, sev_trft.shape[1]):
            if (random.random() < 0.5):
                sev_trft[i][j] = 0
    return sev_trft

def sev_trf_org(t_sev_trf, t_dft_dict, t_chosen_sevs):
    sev_trft = copy.deepcopy(t_sev_trf)#np.zeros((len(chosen_sevs), len(chosen_sevs)))
    for i in range(len(t_chosen_sevs) - 1):
        for j in range(i + 1, len(t_chosen_sevs)):
            i_m = t_chosen_sevs[i]
            j_m = t_chosen_sevs[j]
            if ((i_m, j_m) in list(t_dft_dict.keys())): 
                sev_trft[i][j] = t_dft_dict[(i_m, j_m)]
                if sev_trft[i][j] < 1:
                    sev_trft[i][j] = random.randint(1, 10)
            elif ((j_m, i_m) in list(t_dft_dict.keys())):
                sev_trft[i][j] = t_dft_dict[(j_m, i_m)]
                if sev_trft[i][j] < 1:
                    sev_trft[i][j] = random.randint(1, 10)
                    
            sev_trft[i][j] = int(sev_trft[i][j])
    return sev_trft


def cnt_sev_trf(t_sev_trf):
    cnt = 0
    for i in range(t_sev_trf.shape[1] - 1):
        for j in range(i + 1, t_sev_trf.shape[1]):
            if t_sev_trf[i][j] != 0:
                cnt += 1
    print(cnt)
                


In [330]:
#验证对不对
a = 0
for i in mac_cpu_mem_sev_new:
    a += i[2]
print(a)

a = 0
for i in mac_cpu_mem_sev:
    a += i[2]
print(a)

print()
print(len(mac_cpu_mem_sev_new))
print(len(change))
print(len(changed))
print(len(mac_kinds_sev_kinds_new))
print(len(mac_kinds_num_new))
print()
print(len(mac_cpu_mem_sev))
print(len(mac_kinds_sev_kinds))
print(len(mac_kinds_num))

977
977

186
80
80
186
186

122
122
122


In [310]:
def gen_mac_new(t_i_l, t_mac_kinds_num, t_not_use, t_x, t_mac_cpu_mem_sev, t_mac_ini_sev):
    #t_i_l = random.sample(range(len(sevs_l_r)), t_x) #这是list index
    print(t_i_l)
    #t_i_l的index是mac_cpu_mem_sev_new的
    #for i in t_i_l:
    #    print(sevs_l_r[i])

    for i in t_i_l:
        #t_i_k = sevs_l_r[i]#t_i_k mac_index
        print(t_mac_cpu_mem_sev[i])

    macs = [i for i in t_i_l]#mac_index

    whole_kinds = []
    for i in t_i_l:
        m_index = i
        #第m种物理机的索引编号
        l = sum(t_mac_kinds_num[ : m_index]) 
        r = l + int(t_mac_kinds_num[m_index])
        for m_mk in range(l, r):
            print(m_mk)
            for s in t_mac_ini_sev[m_mk]:
                if s not in whole_kinds:
                    whole_kinds.append(s)

    for i in range(len(whole_kinds)):
        whole_kinds[i] = eval(whole_kinds[i].strip('Service'))

    whole_kinds.sort()
    if_can_use = [i in whole_kinds for i in not_use]

    return macs, whole_kinds, len(whole_kinds), if_can_use

In [331]:
#验证对不对
a = 0
for i in mac_cpu_mem_sev_new:
    a += i[2]
print(a)

a = 0
for i in m_c_m_s_n:
    a += i[2]
print(a)

a = 0
for i in mac_cpu_mem_sev:
    a += i[2]
print(a)

print()
print(len(mac_cpu_mem_sev_new))
print(len(change))
print(len(changed))
print(len(mac_kinds_sev_kinds_new))
print(len(mac_kinds_num_new))
print()
print(len(mac_cpu_mem_sev))
print(len(mac_kinds_sev_kinds))
print(len(mac_kinds_num))
print()
print(len(m_c_m_s_n))
print(len(change_new))
print(len(change_newd_new))
print(len(m_k_s_k_n))
print(len(m_k_n_n))


977
977
977

186
80
80
186
186

122
122
122

186
80
80
186
186


In [322]:
change_new

[(21940.0, 14276.0, 1, 76, 0),
 (21940.0, 14276.0, 1, 2, 1),
 (21940.0, 14276.0, 1, 14, 2),
 (21940.0, 14276.0, 1, 30, 3),
 (21940.0, 14276.0, 3, 77, 4),
 (22269.0, 14490.0, 2, 24, 7),
 (22269.0, 14490.0, 2, 83, 8),
 (22269.0, 14490.0, 2, 80, 9),
 (22269.0, 14490.0, 2, 69, 10),
 (22269.0, 14490.0, 5, 51, 11),
 (10940.0, 5310.0, 2, 61, 19),
 (10940.0, 5310.0, 2, 36, 20),
 (10940.0, 5310.0, 2, 34, 21),
 (10940.0, 5310.0, 2, 22, 22),
 (10940.0, 5310.0, 3, 16, 23),
 (11241.0, 7314.0, 34, 133, 24),
 (11241.0, 7314.0, 34, 139, 25),
 (11241.0, 7314.0, 34, 134, 26),
 (11241.0, 7314.0, 34, 146, 27),
 (11241.0, 7314.0, 37, 141, 28),
 (10986.0, 5333.0, 7, 161, 30),
 (10986.0, 5333.0, 7, 177, 31),
 (10986.0, 5333.0, 7, 148, 32),
 (10986.0, 5333.0, 7, 180, 33),
 (10986.0, 5333.0, 7, 178, 34),
 (10900.0, 14184.0, 3, 70, 37),
 (10900.0, 14184.0, 3, 79, 38),
 (10900.0, 14184.0, 3, 76, 39),
 (10900.0, 14184.0, 3, 68, 40),
 (10900.0, 14184.0, 4, 56, 41),
 (11055.0, 10761.0, 3, 87, 43),
 (11055.0, 10761.

In [268]:
not_use #98 99 100 101 102

[706, 1005, 4873, 5449]

In [327]:
mac_sev_ini_new = copy.deepcopy(mac_ini_sev)
for i in the_same_pos_macs:
    mac_sev_ini_new[i] = mac_sev_ini_new[stop_macs[0]]
    mac_sev_ini_new[i + 1] = mac_sev_ini_new[stop_macs[1]]

In [326]:
stop_macs = [579, 580]
the_same_pos_macs = [559,  564,  569,  574]

In [340]:
macs, whole_kinds, sev_kinds, if_can_use = gen_mac_new([2, 58, 102], m_k_n_n, not_use, 3, m_c_m_s_n, mac_sev_ini_new)
print(if_can_use)
print(sev_kinds)
print()

mac_kinds, whole_mac_nums, per_kind_mac_num, sev_kinds = cmp_para(whole_kinds, macs, m_c_m_s_n)
sub_cpu, sub_mem = sev_index_map(whole_kinds, cpu_s, mem_s)
mac_sev = gen_mac_sev(m_c_m_s_n, m_k_n_n, mac_sev_ini_new, sub_cpu, sub_mem, mac_kinds, whole_mac_nums, per_kind_mac_num, sev_kinds, macs, whole_kinds)
sev_trf = cmp_trf(whole_kinds, sev_kinds)
vm_d = vm_demand(sub_cpu, sub_mem, sev_kinds) 

cnt_sev_trf(sev_trf)
sev_trf_0 = half_sev_trf(sev_trf)
cnt_sev_trf(sev_trf_0)
sev_trf_1 = sev_trf_org(sev_trf_0, dft_dict, whole_kinds)
cnt_sev_trf(sev_trf_1)


per_mac_c_m = total_cpu_mem(macs, m_c_m_s_n, mac_kinds)
each_mac_t_c_m = each_mac_total(whole_mac_nums, per_kind_mac_num, m_c_m_s_n, macs, mac_kinds)

print()
per_sev_kinds = mac_sev.sum(axis = 0)
print(per_sev_kinds.shape[0])
print(vm_d.shape[0])
sev_kinds

now_m_c_m, test_c_m = test_truth(vm_d, mac_sev, sub_cpu, sub_mem, mac_sev_ini_new, cpu_s_ini, mem_s_ini, whole_mac_nums, macs, per_kind_mac_num, each_mac_t_c_m, m_k_n_n, per_sev_kinds)


[2, 58, 102]
(21940.0, 14276.0, 1, 14)
(10946.0, 5313.0, 1, 6)
(10929.0, 21307.0, 5, 94)
2
422
577
578
579
580
581
[False, False, False, False]
102

5147
2607
2607

102
102
机子<MAX
[[ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]]
占有率
[[0.98185962 0.34813673]
 [0.98894573 0.62017692]
 [0.97895507 0.1948186 ]
 [0.97941257 0.23790304]
 [0.20715527 0.02529685]
 [0.20807027 0.02553152]
 [0.95360966 0.17759422]]
vm == sub
True
是不是算对了 个数
True
是不是算对了 个数
True
是不是算对了 个数
True
[[21542.  4970.]
 [10825.  3295.]
 [10699.  4151.]
 [10704.  5069.]
 [ 2264.   539.]
 [ 2274.   544.]
 [10422.  3784.]]
[[21542.  4970.]
 [10825.  3295.]
 [10699.  4151.]
 [10704.  5069.]
 [ 2264.   539.]
 [ 2274.   544.]
 [10422.  3784.]]
机子c m一不一样
True
机子数目一不一样
True
机子vm一不一样
[ True  True]


In [339]:
#总的东西 
savemat("ByteDance_CanShutDown_data_gen_trf" +'_' + str(mac_kinds) + '_' + str(sev_kinds) + ".mat", {"whole_mac_nums" : whole_mac_nums, "mac_kinds": mac_kinds, "mac_kinds_per_num": per_kind_mac_num.reshape((mac_kinds, 1)), "mac_kinds_per_total_cpu_mem": per_mac_c_m, 
                            "vm_total_num": sum(per_sev_kinds), 'vm_kinds' : sev_kinds, 'vm_kinds_per_num' :per_sev_kinds.reshape((per_sev_kinds.shape[0], 1)), 'vm_kinds_per_cpu_mem' :vm_d, 'sev_trf':sev_trf_1 })